# 🚀 Day 2-2: ML을 위한 데이터 전처리 🧹

"Garbage In, Garbage Out." (쓰레기를 넣으면, 쓰레기가 나온다.)

데이터 과학 분야에서 가장 유명한 격언 중 하나입니다. 최고의 모델, 최첨단 알고리즘이 있더라도, 제대로 정제되지 않은 데이터를 사용한다면 결코 좋은 성능을 기대할 수 없습니다. 모델의 성능은 데이터의 품질에 의해 결정되며, **데이터 전처리(Data Preprocessing)** 는 바로 이 '데이터의 품질'을 높이는 핵심적인 과정입니다.

특히 고객 이탈 예측, 스팸 메일 분류 등 **분류(Classification)** 문제에서는 숫자형 데이터와 문자형 데이터가 섞여있고, 값이 빠져있는 경우(결측치)도 흔합니다. 기계는 'Male', 'Female' 같은 문자열을 직접 이해하지 못하며, 값이 비어있으면 계산 과정에서 오류를 일으킵니다.

이번 시간에는 분류 모델을 훈련시키기 전, 데이터를 모델이 이해할 수 있는 깨끗하고 정제된 형태로 만드는 필수적인 기술들을 배웁니다. **결측치 처리**, **범주형 변수 인코딩**, **특성 스케일링**, 그리고 이 모든 과정을 효율적으로 관리하는 **전처리 파이프라인 구축**까지, 모델의 성능을 극대화하는 첫 단추를 함께 꿰어 보겠습니다.

이번 실습에서는 IBM에서 제공한 **통신사 고객 이탈(Telco Customer Churn) 데이터셋**을 사용합니다. 이 데이터는 고객의 요금제 정보, 서비스 사용 패턴 등 다양한 형태의 변수를 포함하고 있어 전처리 기술을 연습하기에 매우 적합합니다.

---

### 1. 데이터 불러오기 및 탐색

가장 먼저, 분석할 데이터를 불러오고 기본적인 구조를 파악해야 합니다. 어떤 변수들이 있고, 데이터 타입은 무엇이며, 비어있는 값은 없는지 확인하는 과정입니다.

#### 💻 코드로 알아보기

[Kaggle: Telco Customer Churn Dataset](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)에서 `WA_Fn-UseC_-Telco-Customer-Churn.csv` 파일을 다운로드하여 실습에 사용합니다.

In [1]:
import pandas as pd
import numpy as np

# 데이터셋 로드
# 자신의 파일 경로에 맞게 수정해주세요.
path = '../datasets/ml/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(path)

# 데이터의 기본적인 정보 확인
print("데이터셋 크기:", df.shape)
print("\n데이터셋 정보:")
df.info()

데이터셋 크기: (7043, 21)

데이터셋 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBil

`df.info()` 결과를 보면, 총 7043개의 데이터와 21개의 컬럼이 있음을 알 수 있습니다. `TotalCharges` 컬럼이 `object` 타입으로 되어있는 점이 특이합니다. 숫자로 보여야 할 것 같은데 왜일까요? 이는 숫자 사이에 공백(space)이 포함된 값이 섞여있기 때문입니다. 이런 값들은 나중에 숫자로 변환하고 결측치로 처리해야 합니다.

---

### 2. 결측치 처리 (Missing Value Imputation)

결측치(Missing Value)는 말 그대로 '값이 빠져있는' 데이터를 의미하며, 보통 `NaN` (Not a Number)으로 표시됩니다. 대부분의 머신러닝 알고리즘은 결측치가 있는 데이터를 처리하지 못하므로, 반드시 훈련 전에 적절한 값으로 채워주어야 합니다.

#### 🧠 개념 이해하기

결측치를 처리하는 방법은 다양하지만, 가장 기본적인 접근법은 특정 값으로 '대체(impute)'하는 것입니다. `scikit-learn`의 `SimpleImputer`는 다음과 같은 간단한 전략을 제공합니다.

* `strategy='mean'`: 해당 열의 **평균값**으로 채웁니다. (수치형 데이터에만 사용 가능)
* `strategy='median'`: 해당 열의 **중앙값**으로 채웁니다. 이상치(outlier)가 많은 경우 평균보다 안정적입니다. (수치형 데이터에만 사용 가능)
* `strategy='most_frequent'`: 해당 열에서 **가장 자주 등장하는 값(최빈값)**으로 채웁니다. (수치형, 범주형 데이터 모두 사용 가능)
* `strategy='constant'`: 우리가 지정하는 **상수 값**으로 채웁니다. (e.g., `fill_value=0` 또는 `fill_value='Unknown'`)

어떤 전략을 선택할지는 데이터의 특성과 분포에 따라 달라집니다.

#### 💻 코드로 알아보기

먼저, 문제가 되었던 `TotalCharges` 컬럼의 공백을 숫자로 변환하고 결측치로 만들겠습니다.

In [2]:
# TotalCharges 컬럼의 공백을 NaN으로 변환 후 float 타입으로 변경
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 결측치가 생성되었는지 확인
print("TotalCharges 결측치 개수:", df['TotalCharges'].isnull().sum())

# SimpleImputer를 사용해 평균값으로 결측치 채우기
from sklearn.impute import SimpleImputer

# 수치형 데이터만 선택하여 새로운 DataFrame 생성
df_numeric = df.select_dtypes(include=np.number)
df_numeric


TotalCharges 결측치 개수: 11


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65
...,...,...,...,...
7038,0,24,84.80,1990.50
7039,0,72,103.20,7362.90
7040,0,11,29.60,346.45
7041,1,4,74.40,306.60


In [4]:
# 'mean' 전략을 사용하는 imputer 객체 생성
imputer = SimpleImputer(strategy='mean')

# imputer를 데이터에 학습(fit)시키고 변환(transform) 적용
# fit: 각 열의 평균값을 계산하여 imputer 객체 내부에 저장
# transform: 저장된 평균값을 사용해 결측치를 채움
df_imputed_array = imputer.fit_transform(df_numeric)
df_imputed_array

array([[0.0000e+00, 1.0000e+00, 2.9850e+01, 2.9850e+01],
       [0.0000e+00, 3.4000e+01, 5.6950e+01, 1.8895e+03],
       [0.0000e+00, 2.0000e+00, 5.3850e+01, 1.0815e+02],
       ...,
       [0.0000e+00, 1.1000e+01, 2.9600e+01, 3.4645e+02],
       [1.0000e+00, 4.0000e+00, 7.4400e+01, 3.0660e+02],
       [0.0000e+00, 6.6000e+01, 1.0565e+02, 6.8445e+03]])

In [5]:

# transform 결과는 numpy 배열이므로 다시 DataFrame으로 변환
df_imputed = pd.DataFrame(df_imputed_array, columns=df_numeric.columns)

print("결측치 처리 후 TotalCharges 정보:")
print(df_imputed['TotalCharges'].describe())
print("결측치 처리 후 결측치 개수:", df_imputed['TotalCharges'].isnull().sum())

결측치 처리 후 TotalCharges 정보:
count    7043.000000
mean     2283.300441
std      2265.000258
min        18.800000
25%       402.225000
50%      1400.550000
75%      3786.600000
max      8684.800000
Name: TotalCharges, dtype: float64
결측치 처리 후 결측치 개수: 0


#### ✏️ 연습문제 1

`SimpleImputer`의 `strategy`를 `'median'`으로 변경하여 `TotalCharges`의 결측치를 **중앙값**으로 채우고, 처리 후 결측치가 없는지 확인해보세요.

In [7]:
# 연습문제 1 코드
imputer_median = SimpleImputer(strategy='median')
df_imputed_median = imputer_median.fit_transform(df_numeric)
df_imputed_median = pd.DataFrame(df_imputed_median, columns=df_numeric.columns)

# 중앙값으로 채워진 후 결측치가 없는지 확인하는 코드를 작성하세요.

print("결측치 처리 후 TotalCharges 정보:")
print(df_imputed_median['TotalCharges'].describe())
print("결측치 처리 후 결측치 개수:", df_imputed_median['TotalCharges'].isnull().sum())

결측치 처리 후 TotalCharges 정보:
count    7043.000000
mean     2281.916928
std      2265.270398
min        18.800000
25%       402.225000
50%      1397.475000
75%      3786.600000
max      8684.800000
Name: TotalCharges, dtype: float64
결측치 처리 후 결측치 개수: 0


---

### 3. 범주형 변수 인코딩 (Categorical Encoding)

머신러닝 모델은 'Male', 'Female'이나 'Yes', 'No' 같은 문자열 데이터를 직접 이해하지 못합니다. 이러한 범주형(Categorical) 데이터를 모델이 처리할 수 있는 숫자 형태로 변환하는 과정을 **인코딩(Encoding)** 이라고 합니다.

#### 3.1 레이블 인코딩 (Label Encoding)

**레이블 인코딩**은 n개의 범주를 0부터 n-1까지의 정수로 변환하는 가장 간단한 방법입니다.

* `'No'` -> `0`, `'Yes'` -> `1`
* `'low'` -> `0`, `'medium'` -> `1`, `'high'` -> `2`

이 방법은 간단하지만, **순서가 없는 명목형 변수(Nominal Variable)**에 적용할 경우 모델이 숫자의 크기(e.g., 2 > 1 > 0)에서 불필요한 관계를 학습할 위험이 있습니다. 따라서 'Yes/No'와 같은 **이진 변수(Binary Variable)** 나, 'low/medium/high'처럼 **순서가 있는 서열 변수(Ordinal Variable)**에 주로 사용됩니다.

##### 💻 코드로 알아보기

`scikit-learn`의 `LabelEncoder`를 사용하여 이진 변수인 `Partner` 컬럼을 인코딩해 보겠습니다.

In [8]:
from sklearn.preprocessing import LabelEncoder

# 'Partner' 컬럼 선택
partner_series = df['Partner']

# LabelEncoder 객체 생성 및 학습/변환
le = LabelEncoder()
partner_encoded = le.fit_transform(partner_series)

print("원본 데이터:", partner_series.unique())
print("인코딩된 데이터:", np.unique(partner_encoded))
print("\n인코딩 클래스 확인:", le.classes_) # 'No'가 0, 'Yes'가 1로 매핑됨

원본 데이터: ['Yes' 'No']
인코딩된 데이터: [0 1]

인코딩 클래스 확인: ['No' 'Yes']


##### ✏️ 연습문제 2

`Dependents` 컬럼에 `LabelEncoder`를 적용하여 0과 1로 변환하고, 어떤 값이 어떤 숫자로 매핑되었는지 확인해보세요.

In [9]:
# 연습문제 2 코드
dependents_series = df['Dependents']

# 여기에 LabelEncoder를 적용하는 코드를 작성하세요.

# LabelEncoder 객체 생성 및 학습/변환
le_for_dept = LabelEncoder()
dependents_encoded = le_for_dept.fit_transform(dependents_series)

print("원본 데이터:", dependents_series.unique())
print("인코딩된 데이터:", np.unique(dependents_encoded))
print("\n인코딩 클래스 확인:", le_for_dept.classes_) # 'No'가 0, 'Yes'가 1로 매핑됨


원본 데이터: ['No' 'Yes']
인코딩된 데이터: [0 1]

인코딩 클래스 확인: ['No' 'Yes']


#### 3.2 원-핫 인코딩 (One-Hot Encoding)

순서가 없는 명목형 변수(e.g., `Contract`: 'Month-to-month', 'One year', 'Two year')에 레이블 인코딩을 적용하면, 모델은 'Two year'(2)가 'Month-to-month'(0)보다 두 배 더 중요하다고 잘못 학습할 수 있습니다.

**원-핫 인코딩**은 이러한 문제를 해결하기 위해 각 범주를 새로운 **'더미 변수(dummy variable)'** 컬럼으로 만들고, 해당 범주에 속하면 1, 아니면 0으로 표시하는 방법입니다.

| Contract (Original) | Contract_Month-to-month | Contract_One year | Contract_Two year |
| :--- | :--- | :--- | :--- |
| Month-to-month | 1 | 0 | 0 |
| One year | 0 | 1 | 0 |
| Two year | 0 | 0 | 1 |
| Month-to-month | 1 | 0 | 0 |

이 방법은 변수 간의 순서 관계가 없다는 것을 명확히 해주지만, 범주의 개수만큼 컬럼이 늘어나는 단점이 있습니다.

##### 💻 코드로 알아보기

`scikit-learn`의 `OneHotEncoder`를 사용하여 `Contract` 컬럼을 인코딩해 보겠습니다. `OneHotEncoder`는 2차원 배열을 입력으로 받으므로 `df[['Contract']]`와 같이 사용해야 합니다.

In [10]:
from sklearn.preprocessing import OneHotEncoder

# 'Contract' 컬럼 선택
contract_df = df[['Contract']]

# OneHotEncoder 객체 생성 및 학습/변환
ohe = OneHotEncoder(sparse_output=False) # sparse_output=False로 해야 numpy 배열로 반환됨
contract_onehot = ohe.fit_transform(contract_df.fillna('Unknown')) # 결측치가 있을 경우를 대비

print("원본 데이터:", contract_df['Contract'].unique())
print("인코딩된 데이터 shape:", contract_onehot.shape)
print("생성된 컬럼명:", ohe.get_feature_names_out())
print("\n인코딩 샘플:\n", contract_onehot[:5])

원본 데이터: ['Month-to-month' 'One year' 'Two year']
인코딩된 데이터 shape: (7043, 3)
생성된 컬럼명: ['Contract_Month-to-month' 'Contract_One year' 'Contract_Two year']

인코딩 샘플:
 [[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


##### ✏️ 연습문제 3

`PaymentMethod` 컬럼에 `OneHotEncoder`를 적용하고, 생성된 더미 변수의 개수(shape)와 컬럼명을 출력해보세요.

In [11]:
# 연습문제 3 코드
payment_df = df[['PaymentMethod']]

# OneHotEncoder 객체 생성 및 학습/변환
ohe = OneHotEncoder(sparse_output=False) # sparse_output=False로 해야 numpy 배열로 반환됨
payment_onehot = ohe.fit_transform(payment_df.fillna('Unknown')) # 결측치가 있을 경우를 대비

print("원본 데이터:", payment_df['PaymentMethod'].unique())
print("인코딩된 데이터 shape:", payment_onehot.shape)
print("생성된 컬럼명:", ohe.get_feature_names_out())

원본 데이터: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
인코딩된 데이터 shape: (7043, 4)
생성된 컬럼명: ['PaymentMethod_Bank transfer (automatic)'
 'PaymentMethod_Credit card (automatic)' 'PaymentMethod_Electronic check'
 'PaymentMethod_Mailed check']


---

### 4. 특성 스케일링 (Feature Scaling)

`tenure`(가입 기간), `MonthlyCharges`(월 요금)와 같이 값의 범위(scale)가 다른 숫자형 변수들이 있을 때, 스케일이 큰 변수가 모델에 더 큰 영향을 미치는 것을 방지하기 위해 모든 변수의 범위를 비슷하게 맞춰주는 작업이 **특성 스케일링**입니다.

이는 특히 거리를 기반으로 하는 알고리즘(KNN), 경사 하강법을 사용하는 알고리즘(로지스틱 회귀, SVM, 신경망)에서 성능에 큰 영향을 줍니다.

#### 4.1 표준화 (Standardization)

**표준화**는 각 특성의 **평균을 0, 표준편차를 1**로 변환하는 방법입니다. 이상치에 비교적 덜 민감하며, 데이터가 정규 분포를 따를 때 효과적입니다. `StandardScaler`를 사용합니다.

$$z = \frac{(x - \mu)}{\sigma}$$

* $x$: 원본 데이터
* $\mu$: 평균
* $\sigma$: 표준편차

##### 💻 코드로 알아보기

앞서 결측치를 채운 `df_imputed` 데이터의 수치형 변수들에 `StandardScaler`를 적용해 보겠습니다.

In [12]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
scaler = StandardScaler()

# 스케일러 학습 및 변환
df_scaled_standard = scaler.fit_transform(df_imputed)
df_scaled_standard = pd.DataFrame(df_scaled_standard, columns=df_imputed.columns)

print("--- 스케일링 전 ---")
print(df_imputed.describe())
print("\n--- StandardScaler 적용 후 ---")
print(df_scaled_standard.describe())

--- 스케일링 전 ---
       SeniorCitizen       tenure  MonthlyCharges  TotalCharges
count    7043.000000  7043.000000     7043.000000   7043.000000
mean        0.162147    32.371149       64.761692   2283.300441
std         0.368612    24.559481       30.090047   2265.000258
min         0.000000     0.000000       18.250000     18.800000
25%         0.000000     9.000000       35.500000    402.225000
50%         0.000000    29.000000       70.350000   1400.550000
75%         0.000000    55.000000       89.850000   3786.600000
max         1.000000    72.000000      118.750000   8684.800000

--- StandardScaler 적용 후 ---
       SeniorCitizen        tenure  MonthlyCharges  TotalCharges
count   7.043000e+03  7.043000e+03    7.043000e+03  7.043000e+03
mean   -4.842546e-17 -2.421273e-17   -6.406285e-17  8.070910e-17
std     1.000071e+00  1.000071e+00    1.000071e+00  1.000071e+00
min    -4.399165e-01 -1.318165e+00   -1.545860e+00 -9.998503e-01
25%    -4.399165e-01 -9.516817e-01   -9.725399e-01 -8.3

결과를 보면 모든 컬럼의 평균(mean)이 거의 0에 가깝고, 표준편차(std)가 1로 변환된 것을 확인할 수 있습니다.

#### 4.2 정규화 (Normalization)

**정규화**는 각 특성의 값을 **0과 1 사이의 범위**로 변환하는 방법입니다. 데이터의 분포를 유지하면서 스케일을 조정하고 싶을 때 유용합니다. `MinMaxScaler`를 사용합니다.

$$X_{norm} = \frac{(X - X_{min})}{(X_{max} - X_{min})}$$

##### 💻 코드로 알아보기

`df_imputed` 데이터에 `MinMaxScaler`를 적용해 보겠습니다.

In [13]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 객체 생성
minmax_scaler = MinMaxScaler()

# 스케일러 학습 및 변환
df_scaled_minmax = minmax_scaler.fit_transform(df_imputed)
df_scaled_minmax = pd.DataFrame(df_scaled_minmax, columns=df_imputed.columns)

print("\n--- MinMaxScaler 적용 후 ---")
print(df_scaled_minmax.describe())


--- MinMaxScaler 적용 후 ---
       SeniorCitizen       tenure  MonthlyCharges  TotalCharges
count    7043.000000  7043.000000     7043.000000   7043.000000
mean        0.162147     0.449599        0.462803      0.261309
std         0.368612     0.341104        0.299403      0.261366
min         0.000000     0.000000        0.000000      0.000000
25%         0.000000     0.125000        0.171642      0.044245
50%         0.000000     0.402778        0.518408      0.159445
75%         0.000000     0.763889        0.712438      0.434780
max         1.000000     1.000000        1.000000      1.000000


결과를 보면 모든 컬럼의 최소값(min)이 0, 최대값(max)이 1로 변환된 것을 확인할 수 있습니다.

#### ✏️ 연습문제 4

`df_imputed` 데이터프레임에서 `tenure`와 `TotalCharges` 두 컬럼만 선택하여 `StandardScaler`로 표준화를 진행하고, 변환된 데이터의 첫 5행을 출력해보세요.

In [14]:
# 연습문제 4 코드
StandardScaler().fit_transform(df_imputed[['tenure', 'TotalCharges']])[:5]

array([[-1.27744458, -0.99497138],
       [ 0.06632742, -0.17387565],
       [-1.23672422, -0.96039939],
       [ 0.51425142, -0.19540036],
       [-1.23672422, -0.94119274]])

---

### 5. 데이터 불균형 처리 (Handling Imbalanced Data)

분류 문제에서 타겟 변수의 클래스 비율이 심하게 차이 나는 경우를 **데이터 불균형(Imbalanced Data)** 이라고 합니다. 예를 들어, 고객 이탈 예측에서 이탈 고객(Yes)이 10%, 비이탈 고객(No)이 90%라면, 모델은 무조건 'No'라고만 예측해도 90%의 정확도를 얻게 됩니다. 이는 우리가 원하는 소수 클래스(이탈 고객)를 제대로 예측하지 못하는 결과를 낳습니다.

이를 해결하는 방법 중 하나가 **오버샘플링(Oversampling)** 이며, 가장 대표적인 기법이 **SMOTE(Synthetic Minority Over-sampling Technique)** 입니다.

#### 🧠 개념 이해하기

**SMOTE**는 단순히 소수 클래스의 데이터를 복제하는 것이 아니라, 소수 클래스 데이터 포인트들 사이의 공간에 **'가상의(Synthetic)' 데이터를 생성**하여 데이터 수를 늘리는 방식입니다.

1.  소수 클래스에서 임의의 데이터 포인트($A$)를 선택합니다.
2.  그 데이터와 가장 가까운 k개의 이웃($B, C, ...$)을 찾습니다.
3.  $A$와 이웃들 사이의 직선상에 임의의 점을 찍어 새로운 데이터 포인트($A'$)를 생성합니다.
4.  이 과정을 소수 클래스와 다수 클래스의 데이터 수가 비슷해질 때까지 반복합니다.

**중요**: SMOTE와 같은 오버샘플링 기법은 반드시 **훈련(Train) 데이터에만 적용**해야 합니다. 테스트(Test) 데이터에 적용하면 모델의 성능을 부풀리게 되어 올바른 평가를 할 수 없습니다.

#### 💻 코드로 알아보기

`imblearn` 라이브러리를 설치(`pip install imbalanced-learn`)하고 SMOTE를 적용해 보겠습니다.

In [ ]:
!pip install imbalanced-learn

데이터 로드

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
df = pd.read_csv("../datasets/ml/creditcardfraud/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [18]:
# 기초통계량 확인
df[['Class']].describe()

,Class
count,284807.000000
mean,0.001727
std,0.041527
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


시각화

In [19]:
import plotly.express as px # 시각화
class_counts = df.Class.value_counts().reset_index()
class_counts.columns = ['Class', 'Count']
fig = px.bar(class_counts, x='Class', y='Count', title='Class Counts', labels={'Class': '분류', 'Count': '개수'}, text='Count')
fig.update_layout(yaxis=dict(title='클래스별 개수'), width=600, height=400)
fig.show()

In [20]:
import sys
import os
# 프로젝트 루트 디렉토리를 sys.path에 추가
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root_path not in sys.path:
    sys.path.append(project_root_path)
# 커스텀 라이브러리 로드
from lib.visualize import TSNEVisualizer

데이터 2차원 평면 분포 시각화 (n=3000)

In [21]:
# 데이터를 x와 y로 분할
X = df.drop('Class', axis=1)
y = df['Class']
# 불균형 데이터 랜덤 샘플링
X_random_sampled = X.sample(3000)
y_random_sampled = y.iloc[X_random_sampled.index]
# 시각화
TSNEVisualizer.visualize(X_random_sampled, y_random_sampled)

언더샘플링 예시

In [23]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)
print("--- 랜덤 언더샘플링 적용 전 ---")
print("Resampled X_train shape:", X.shape)
print("Resampled y_train value counts:\n", pd.Series(y).value_counts())

print("--- 랜덤 언더샘플링 적용 후 ---")
print("Resampled X_train shape:", X_resampled.shape)
print("Resampled y_train value counts:\n", pd.Series(y_resampled).value_counts())

--- 랜덤 언더샘플링 적용 전 ---
Resampled X_train shape: (284807, 30)
Resampled y_train value counts:
 Class
0    284315
1       492
Name: count, dtype: int64
--- 랜덤 언더샘플링 적용 후 ---
Resampled X_train shape: (984, 30)
Resampled y_train value counts:
 Class
0    492
1    492
Name: count, dtype: int64


전용 커스텀 클래스로 진행

In [24]:
from lib.preperate import ImbalancedDataAnalyzer
# Down sampling : 적은 쪽 클래스는 그대로, 많은 쪽 클래스는 랜덤 샘플링(적은쪽 클래수 수 만큼)
analyzer = ImbalancedDataAnalyzer(X, y)
X_rus, y_rus = analyzer.random_undersample()

print("Resampled y_train value counts:\n", pd.Series(y_rus).value_counts())

Resampled y_train value counts:
 Class
0    492
1    492
Name: count, dtype: int64


In [25]:
X_smote, y_smote = analyzer.smote()
print("--- SMOTE 적용 후 ---")
print("Resampled X_train shape:", X_smote.shape)
print("Resampled y_train value counts:\n", pd.Series(y_smote).value_counts())

--- SMOTE 적용 후 ---
Resampled X_train shape: (568630, 30)
Resampled y_train value counts:
 Class
0    284315
1    284315
Name: count, dtype: int64


SMOTE 적용 후, 소수 클래스(1)의 데이터 수가 다수 클래스(0)와 동일하게 증가한 것을 볼 수 있습니다.

#### ✏️ 연습문제 5

SMOTE가 적용된 데이터의 클래스 비율을 `value_counts(normalize=True)`를 사용하여 백분율로 확인해보세요.

In [26]:
# 연습문제 5 코드
X_train_resampled, y_train_resampled = analyzer.smote()
print(pd.Series(y_train_resampled).value_counts(normalize=True))

Class
0    0.5
1    0.5
Name: proportion, dtype: float64


#### ✏️ 연습문제 6

`ImbalancedDataAnalyzer` 클래스의 다른 오버샘플링 기법들(ADASYN, SMOTE+Tomek, SMOTE+ENN)을 사용하여 데이터를 샘플링한 후, 각각의 결과에서 3000개씩 랜덤하게 선택하여 t-SNE를 사용한 2차원 시각화를 수행해보세요.

1. ADASYN, SMOTE+Tomek, SMOTE+ENN 각각으로 오버샘플링 수행
2. 각 샘플링 결과에서 3000개씩 랜덤 선택 (random_state=42)
3. t-SNE로 2차원 시각화
4. 각 기법별로 클래스 분포와 데이터 분포를 시각적으로 비교

힌트: 
- `lib.visualize.TSNEVisualizer` 사용
- 3000개 샘플링을 위해 `sklearn.utils.resample` 활용


In [ ]:
# 연습문제 6 정답
from lib.preperate import ImbalancedDataAnalyzer
from lib.visualize import TSNEVisualizer
from sklearn.utils import resample

analyzer = ImbalancedDataAnalyzer(X, y)

X_adasyn, y_adasyn = analyzer.adasyn()
print(f"ADASYN 샘플링 후 데이터 크기: {X_adasyn.shape}")
print(f"ADASYN 클래스 분포:\n{pd.Series(y_adasyn).value_counts()}")

X_smote_tomek, y_smote_tomek = analyzer.smote_tomek()
print(f"\nSMOTE+Tomek 샘플링 후 데이터 크기: {X_smote_tomek.shape}")
print(f"SMOTE+Tomek 클래스 분포:\n{pd.Series(y_smote_tomek).value_counts()}")

X_smote_enn, y_smote_enn = analyzer.smote_enn()
print(f"\nSMOTE+ENN 샘플링 후 데이터 크기: {X_smote_enn.shape}")
print(f"SMOTE+ENN 클래스 분포:\n{pd.Series(y_smote_enn).value_counts()}")

df_adasyn = pd.DataFrame(X_adasyn, columns=X.columns)
df_adasyn['target'] = y_adasyn
X_adasyn_sample, y_adasyn_sample = resample(df_adasyn.drop('target', axis=1), 
                                           df_adasyn['target'], 
                                           n_samples=3000, 
                                           random_state=42)

df_smote_tomek = pd.DataFrame(X_smote_tomek, columns=X.columns)
df_smote_tomek['target'] = y_smote_tomek
X_smote_tomek_sample, y_smote_tomek_sample = resample(df_smote_tomek.drop('target', axis=1), 
                                                     df_smote_tomek['target'], 
                                                     n_samples=3000, 
                                                     random_state=42)

df_smote_enn = pd.DataFrame(X_smote_enn, columns=X.columns)
df_smote_enn['target'] = y_smote_enn
X_smote_enn_sample, y_smote_enn_sample = resample(df_smote_enn.drop('target', axis=1), 
                                                 df_smote_enn['target'], 
                                                 n_samples=3000, 
                                                 random_state=42)

print(f"\n3000개 샘플링 후:")
print(f"ADASYN 클래스 분포: {pd.Series(y_adasyn_sample).value_counts()}")
print(f"SMOTE+Tomek 클래스 분포: {pd.Series(y_smote_tomek_sample).value_counts()}")
print(f"SMOTE+ENN 클래스 분포: {pd.Series(y_smote_enn_sample).value_counts()}")

TSNEVisualizer.visualize(X_adasyn_sample, y_adasyn_sample, 'ADASYN t-SNE 시각화')
TSNEVisualizer.visualize(X_smote_tomek_sample, y_smote_tomek_sample, 'SMOTE+Tomek t-SNE 시각화')
TSNEVisualizer.visualize(X_smote_enn_sample, y_smote_enn_sample, 'SMOTE+ENN t-SNE 시각화')

ADASYN 샘플링 후 데이터 크기: (568613, 30)
ADASYN 클래스 분포:
Class
0    284315
1    284298
Name: count, dtype: int64


---

### 6. 전처리 파이프라인 구축 (Building a Preprocessing Pipeline)

지금까지 배운 결측치 처리, 인코딩, 스케일링을 각 단계별로 따로 적용하는 것은 번거롭고, 테스트 데이터에 동일한 절차를 반복하다 실수를 유발하기 쉽습니다. 

특히, 훈련 데이터에서 학습한 스케일러나 인코더를 테스트 데이터에 그대로 적용해야 하는데(Data Leakage 방지), 이 과정을 잊어버릴 수도 있습니다.

`scikit-learn`의 `Pipeline`과 `ColumnTransformer`는 이러한 전처리 과정들을 하나의 '작업 흐름'으로 묶어주는 강력한 도구입니다.

* `ColumnTransformer`: **서로 다른 컬럼 그룹에 서로 다른 변환을 적용**할 수 있게 해줍니다. (e.g., 수치형 컬럼에는 결측치 처리+스케일링, 범주형 컬럼에는 결측치 처리+원핫인코딩)
* `Pipeline`: 여러 단계의 변환 과정과 마지막의 모델까지를 **하나의 단일 객체**로 묶어줍니다.

#### 💻 코드로 알아보기

수치형 변수와 범주형 변수에 각각 다른 전처리 방식을 적용하고, 최종적으로 로지스틱 회귀 모델을 학습시키는 전체 파이프라인을 구축해 보겠습니다.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 원본 데이터 다시 로드 및 분리
path = '../datasets/ml/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(path)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 1. 컬럼 그룹 정의
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()
# customerID는 모델링에 불필요하므로 제거
if 'customerID' in numeric_features:
    numeric_features.remove('customerID')

# 2. 각 그룹에 대한 전처리 파이프라인 정의
# 수치형 변수 파이프라인: 결측치 중앙값 대체 -> 표준화
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 변수 파이프라인: 결측치 최빈값 대체 -> 원핫인코딩
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 테스트 데이터에 없는 범주가 나와도 에러 방지
])

# 3. ColumnTransformer로 두 파이프라인 통합
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 4. 최종 파이프라인 구축: 전처리기 + 모델
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear'))
])

# 5. 파이프라인 학습
model_pipeline.fit(X_train, y_train)

# 6. 파이프라인으로 예측 및 평가
accuracy = model_pipeline.score(X_test, y_test)
print(f"파이프라인 모델의 정확도: {accuracy:.4f}")

파이프라인 모델의 정확도: 0.8041


이렇게 파이프라인을 구축하면 `fit` 한 번으로 모든 전처리 과정과 모델 학습이 한 번에 이루어지며, `predict`나 `score`를 호출하면 내부적으로 테스트 데이터에 동일한 전처리 과정을 자동으로 적용해주어 매우 편리하고 안전합니다.

#### ✏️ 연습문제 7

위 `model_pipeline`에서 `StandardScaler` 대신 `MinMaxScaler`를 사용하도록 `numeric_transformer`를 수정하여 새로운 파이프라인을 만들고, 정확도를 비교해보세요.

In [ ]:
# 연습문제 7 코드
# numeric_transformer를 수정한 후, 전체 파이프라인을 다시 정의하고 학습/평가하는 코드를 작성하세요.

#### ✏️ 연습문제 8 : 심부전 예측을 위한 전처리 파이프라인

이번에는 Heart Failure Records 데이터셋을 사용하여 환자의 심부전으로 인한 사망 여부(DEATH_EVENT)를 예측하는 모델을 만들어 보겠습니다. 

이 데이터셋은 수치형 변수와 이진(0 또는 1) 범주형 변수로 구성되어 있습니다.

* 처리 가이드:

    Heart Failure 데이터셋을 로드하고 특성(X)과 타겟(y)을 분리합니다.

    수치형 변수와 범주형(이진) 변수를 식별합니다.
    
    ColumnTransformer를 사용하여 수치형 변수에는 StandardScaler를, 범주형 변수에는 OneHotEncoder를 적용하는 전처리기를 만듭니다.

    전처리기와 RandomForestClassifier 모델을 Pipeline으로 결합합니다.

    생성된 파이프라인을 훈련시키고, 테스트 데이터에 대한 정확도를 평가합니다.

데이터 준비

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 데이터셋 로드 (웹에서 직접 로드)
url = '../datasets/ml/heart-failure/heart_failure_clinical_records_dataset.csv'
df_heart = pd.read_csv(url)
df_heart.head()

코드 작성

In [ ]:
# 특성(X)과 타겟(y) 분리

# 훈련/테스트 데이터 분리

# [문제 1] 수치형 특성과 범주형(이진) 특성의 컬럼명을 리스트로 정의하세요.
# 팁: 이 데이터셋의 범주형 특성은 모두 0과 1로 되어 있습니다. (e.g., 'anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking')
# 나머지 컬럼은 수치형으로 간주할 수 있습니다.
numeric_features = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
categorical_features = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

# [문제 2] ColumnTransformer를 사용하여 전처리기를 만드세요.
# 수치형 특성에는 StandardScaler를 적용하세요.
# 범주형 특성에는 OneHotEncoder를 적용하세요. (이진 변수지만 연습을 위해 적용)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', ?, ?),
        ('cat', ?, ?)
    ])

# [문제 3] Pipeline을 사용하여 전처리기와 RandomForestClassifier 모델을 연결하세요.
# RandomForestClassifier의 random_state는 42로 설정하세요.
model_pipeline = Pipeline(steps=[
    ('preprocessor', ?),
    ('classifier', ?(random_state=42))
])

# [문제 4] 생성한 파이프라인을 훈련 데이터로 학습시키세요.
# ?

# [문제 5] 학습된 파이프라인으로 테스트 데이터의 예측을 수행하고 정확도를 계산하여 출력하세요.
# y_pred = ?
# accuracy = ?
# print(f"심부전 예측 모델 파이프라인의 정확도: {accuracy:.4f}")